# Indeed Scraper
http://mattdturner.com/wordpress/2015/04/scrape-keywords-from-indeed-com-job-postings-2/

In [1]:
from bs4 import BeautifulSoup
import urllib
import re
from time import sleep
from collections import Counter
from nltk.corpus import stopwords
import pandas as pd
%matplotlib inline
import matplotlib.pylab as plt
from matplotlib.backends.backend_pdf import PdfPages
plt.rcParams['figure.figsize'] = (10.0, 8.0)

/Users/macbook/anaconda/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [2]:
# Define the city, state, and job
city = 'Philadelphia'
state = 'PA'
job_title = 'Data Scientist'

#### Define a function that will take the url and pull out the text of the main body as a list of strings.

In [3]:
def clean_the_html(url):
    # First try to download the html file
    try:
        html = urllib.urlopen(url)
    except:
        return
    
    #print url
    
    # Open html in BeautifulSoup
    soup = BeautifulSoup(html)
        
    # Extract everything within the tags
    text = soup.findAll('body')
    word_list = ''
    for line in text:
        word_list = ' '.join([word_list,line.get_text(' ',strip=True).lower()])
    
    # Remove non text characters from list
    word_list = re.sub('[^a-zA-Z+3]',' ', word_list)

    list_of_words = word_list.encode('ascii','ignore').split()
                   
    stop_words = set(stopwords.words("english"))
    additional_stop_words = ['webfont','limited','saved','disability',\
                             'desirable','nreum','skills','net','+','k',\
                            'above','it','end','excellent','join','want',\
                            'how','well','sets','like','page','home','demonstrated',\
                            'other','re','size','etc','gettime','work','ms',\
                            'zqdxyrmad','description','value','re','transactionname',\
                            'education','daylight','highly','bodyrendered',\
                            'amazon','new','bam','techniques','com',city.lower(),\
                            state.lower(),'min','need','email','job','content','features',\
                            'service','wa','id','modern','looking','eastern',\
                            'qualifications','teams','based','false','times',\
                            'software','career','ability','platform','years','data',\
                            'date','product','team','time','agent','information',\
                            'methods','candidate','customers','back','info','scientist',\
                            'experience','apply','us','engineering','learning',\
                            'strong','business','design','title','large','e','document',\
                            'science','company','location','field','communication',\
                            'customer','tools','used','research','model',\
                            'opportunity','online','including','degree',\
                            'preferred','across','beacon','using','friend','function',\
                            'position','window','role','3','written','build',\
                            'presentation','getelementbyid','technical','posted',\
                            'newrelic','decision','log','errorbeacon','solutions',\
                            'applicationtime','enable','responsibilities',\
                            'models','applicationid','complex','licensekey',\
                            'high','browser','d','nr','develop','please',\
                            'selection','queuetime','cookies','icimsaddonload',\
                            'computer','icims','scientists','great','returning',\
                            'systems','writing','united','working','iframe',\
                            'analyses','applications','try','related',\
                            'states','languages','yghvbe','language','one',\
                            'site','llc,','category','personalized','knowledge']
    
    # Remove words from list
    truncated_list = [w for w in list_of_words if not (w in stop_words or \
                      w in additional_stop_words)]
    
    truncated_set = set(truncated_list)
    truncated_list = list(truncated_set)
        
    return truncated_list

#### Define a function to generate a list of urls for a given search (i.e., ‘Data Scientist’). Each search result page has 10 non-sponsored links. Search the first url for ‘Jobs # to # of ###’ in order to determine how many iterations to perform.

In [4]:
def gen_url_list(city,state,job_name):
    base_url = 'http://www.indeed.com/'
    
    job_term = re.sub(' ','+',job_name.lower())
    
    search_url = ''.join([base_url,'jobs?q=',job_term,'&l=',city,'%2C+',state])
    
    try:
        html = urllib.urlopen(search_url)
    except:
        return
    
    soup = BeautifulSoup(html)
    
    total_jobs = soup.find(id = 'searchCount').string.encode('utf-8')
    job_nums = int([int(s) for s in total_jobs.split() if s.isdigit()][-1]/10)
    print (total_jobs)

    job_URLS = []
    for i in range(job_nums+1):
        if i % 10 == 0:
            print (i)
        page_url = ''.join([base_url,job_term,'&1=',city,'%2C+',state,\
                            '&start=',str((i+1)*10)])
        html = urllib.urlopen(search_url)
        
        soup = BeautifulSoup(html)
        
        job_link_area = soup.findAll('h2',{'class':'jobtitle'})

        for link in job_link_area:
            match_href = re.search('',str(link))
            if match_href:
                job_URLS.append([base_url + match_href.group(1)])

    return job_URLS

In [5]:
def job_posting_analysis(url_list):
    job_skills = []
    count = 0
    for url in url_list:
        count += 1
        if count % 10 == 1:
            print (count)
        
        posting_keywords = clean_the_html(url[0])
        if posting_keywords:
            job_skills.append(posting_keywords)
        sleep(0.5)
        
    return job_skills


In [6]:
print ('Crawl indeed.com for ' + city + ', ' + state + ' ' + job_title + \
' postings and generate a list of all of the job posting links')

url_list = gen_url_list(city,state,job_title)

print ("Given the job posting links, pull out the keywords for each posting")

job_skills = job_posting_analysis(url_list)

Crawl indeed.com for Philadelphia, PA Data Scientist postings and generate a list of all of the job posting links
Given the job posting links, pull out the keywords for each posting


TypeError: 'NoneType' object is not iterable